# Projet de Machine Learning  UMONS 2024-2025

### Thème : Prédiction du score de Macron aux 2nd Tour des éléctions 2022

----

# Configuration et Installation des Dépendances

## a. Import des Bibliothèques/dépendances


In [37]:
# @title manipulation des vecteurs
import pandas as pd
import numpy as np


In [38]:
# @title création des graphiques
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick


In [91]:
# @title prétraitement des données en masse
from sklearn.preprocessing import OneHotEncoder, RobustScaler , FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


In [40]:
# @title selection des features
from sklearn.feature_selection import VarianceThreshold


In [41]:
# @title sélection du meilleur modèle
%pip install optuna
import optuna
from sklearn.model_selection import train_test_split, KFold, cross_val_score , RepeatedKFold

In [42]:
# @title fonction de score et evaluation
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score ,mean_absolute_error

In [43]:
# @title initialisation des modèles
from sklearn.linear_model import ElasticNet, LassoCV, Ridge , Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor


In [44]:
# @title options sytèmes
from sys import modules as sys_modules
import os
import sys
import joblib
import warnings
import os
import glob
import re
from IPython.display import Markdown, display

RANDOM_STATE = 42


## b. Configuration


In [45]:
#Pour ignorer les warnings
warnings.filterwarnings('ignore')

# configuration des graphiques
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette('Set2')

# Pour une meilleure lisibilité dans le notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.3f}'.format)

print("chargement des bibliothèques terminé")

chargement des bibliothèques terminé


# 1. EXPLORATION DES DONNEES


Le but ici c'est d'essayer de comprendre  les données , c'est pouvoir repondre aux questions :
* Quelles sont les données visiblement non-pertinentes ?
* Detecter les outliers ?
* Vérifier le taux de valeurs manquantes
* regrouper les informations en indices synthétiques

----

## 1.1 Chargement des données

In [46]:
# @title chemin vers le dossier dataset : dossier contenant toute les données à utiliser
if 'google.colab' in sys_modules :
  from google.colab import drive
  drive.mount('/content/drive')
  data_path = "/content/drive/MyDrive/Colab_Notebooks/ProjetML/src/datasets"
else:
  data_path = "datasets"

print("debut du chargement des données ... .. ... ..")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
debut du chargement des données ... .. ... ..


In [47]:
# @title Chargement des données de d'entrainement et de test
result_train = pd.read_csv( os.path.join(data_path,"results_train.csv") , sep = ',',encoding='utf-8')
result_test = pd.read_csv( os.path.join(data_path,"results_test.csv") , sep = ',',encoding='utf-8')

res_train_df = result_train.copy()
res_test_df = result_test.copy()

#### Données additionnelles

In [48]:
# @title Niveau de vie
niveau_vie = pd.read_excel(os.path.join(data_path, "Niveau_de_vie_2013_a_la_commune.xlsx"))
niveau_vie_df = niveau_vie.copy()

In [49]:
# @title Communes de France
communes_france = pd.read_csv(os.path.join(data_path, "communes-france-2022.csv"), sep=',', encoding='utf-8')
communes_df = communes_france.copy()

In [50]:
# @title Données d'âge
age_insee = pd.read_excel(os.path.join(data_path, "age-insee-2020.xlsx"))
age_df = age_insee.copy()

In [51]:
# @title Données diverses INSEE
insee_divers = pd.read_excel(os.path.join(data_path, "MDB-INSEE-V2.xls"))
insee_divers_df = insee_divers.copy()

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [52]:
print("chargement des données terminé !! ")

chargement des données terminé !! 


## 1.2 Pré-Analyse  et Pré-Selection(visuelle) des Features (colonnes)

In [53]:
# @title colonne cible
target = '% Voix/Ins'

In [55]:
# @title Utilitaires
THRESHOLD = 60 # @param {"type":"integer"}
IDs = [] # listes des identifiants dans chaque datafame
def sep():
    """Affiche une ligne de séparation"""
    print("\n" + "-"*180 + "\n")

def sub(l1,l2):
  """Retourne la liste des éléments de l1 qui ne sont pas dans l2"""
  return [x for x in l1 if x not in l2]

def get_columns_above_missing_threshold(df, threshold:int=THRESHOLD):
    """
    Identifie les colonnes ayant un pourcentage de valeurs manquantes supérieur au seuil spécifié.

    Args:
        df (pandas.DataFrame): Le DataFrame à analyser
        threshold (float): Le seuil en pourcentage (entre 0 et 100) au-delà duquel une colonne est considérée
                          comme ayant trop de valeurs manquantes. Par défaut : 50

    Returns:
        list: Liste des noms de colonnes dont le pourcentage de valeurs manquantes dépasse le seuil,
              triée par pourcentage décroissant
    """
    # Vérification que le seuil est valide
    if not 0 <= threshold <= 100:
        raise ValueError("Le seuil doit être compris entre 0 et 100")

    # Calcul du pourcentage de valeurs manquantes par colonne
    missing_percentages = (df.isnull().sum() / len(df)) * 100

    # Sélection des colonnes dépassant le seuil
    columns_above_threshold = missing_percentages[missing_percentages > threshold]

    # Tri par pourcentage décroissant
    columns_above_threshold = columns_above_threshold.sort_values(ascending=False)

    # Création d'un DataFrame avec les colonnes et leurs pourcentages
    """result_df = pd.DataFrame({
        'Colonne': columns_above_threshold.index,
        'Pourcentage de valeurs manquantes': columns_above_threshold.values
    })"""

    return  columns_above_threshold.index.to_list() , pd.DataFrame(columns_above_threshold)

def visualise(df):
  print(f"forme : {df.shape}")
  sep()

def all_columns(df, res=True):
  all_col = df.columns.tolist()
  print(f"les colonnes sont : \n{df.columns}")
  sep()
  print(f" il y'a {len(all_col)} colonnes dans le dataframe")
  sep()
  if res:
    return all_col
  else:
    return None


def get_analyse(data , id , col_to_drop , res =True):
    df= data.copy()
    all_col = df.columns.tolist()
    col_to_keep = sub(all_col,col_to_drop)
    to_holes,_ = get_columns_above_missing_threshold(df[col_to_keep])
    col_to_keep2 = sub(col_to_keep,to_holes)
    IDs.append(id)
    print(f"colonne identifiant : '{id}' ")
    sep()
    print(f" colonnes avec > {THRESHOLD} % de valeurs manquantes \n{to_holes}")
    sep()
    print(f"les colonnes à supprimer sont :\n{col_to_drop}  \net  {to_holes} ")
    sep()
    print(f"\non conserve les colonnes  :\n{col_to_keep2} ")
    sep()
    if res:
      return col_to_keep2 , to_holes
    else:
      return None



def write_markdown_conclusion(id_colonne, colonnes_manquantes, colonnes_supprimer, colonnes_conserver):
    """
    Génère une section Markdown pour la conclusion de l'analyse d'un DataFrame.

    Args:
        id_colonne (str): Nom de la colonne d'identification.
        colonnes_manquantes (list): Liste des colonnes avec un fort taux de valeurs manquantes.
        colonnes_supprimer (list): Liste des colonnes à supprimer.
        colonnes_conserver (list): Liste des colonnes à conserver.
    """

    markdown_text = f"""
#### Conclusion

* _Identifiant_ : ``'{id_colonne}'``

* _Colonnes avec plus de {THRESHOLD}% de valeurs manquantes_ : ``{colonnes_manquantes}``

* _Colonnes à supprimer_ : ``{colonnes_supprimer}``

* _Colonnes à conserver_ : ``{colonnes_conserver}``

----
"""
    display(Markdown(markdown_text))


### results_train

In [56]:
# @title visualisation
visualise(res_train_df)

res_train_df.head(3)

forme : (20892, 32)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



,CodeINSEE,Libellé du département,Libellé de la commune,Etat saisie,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,N°Panneau,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,01006,Ain,Ambléon,Complet,103,19,18.450,84,81.550,12,11.650,14.290,0,0.000,0.000,72,69.900,85.710,1,M,MACRON,Emmanuel,45,43.690,62.500,2,F,LE PEN,Marine,27,26.210,37.500
1,01009,Ain,Andert-et-Condon,Complet,280,73,26.070,207,73.930,22,7.860,10.630,3,1.070,1.450,182,65.000,87.920,1,M,MACRON,Emmanuel,102,36.430,56.040,2,F,LE PEN,Marine,80,28.570,43.960
2,01010,Ain,Anglefort,Complet,792,185,23.360,607,76.640,31,3.910,5.110,8,1.010,1.320,568,71.720,93.570,1,M,MACRON,Emmanuel,227,28.660,39.960,2,F,LE PEN,Marine,341,43.060,60.040


In [57]:
# @title affichage des colonnes
all_col1 = all_columns(res_train_df)

les colonnes sont : 
Index(['CodeINSEE', 'Libellé du département', 'Libellé de la commune',
       'Etat saisie', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants',
       '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls',
       '% Nuls/Ins', '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot',
       'N°Panneau', 'Sexe', 'Nom', 'Prénom', 'Voix', '% Voix/Ins',
       '% Voix/Exp', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32'],
      dtype='object')

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 il y'a 32 colonnes dans le dataframe

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [58]:
# @title présélection
# colonnes clairement non informatives à supprimer
col_to_drop1 = ['Unnamed: 27' ,'Unnamed: 26' , 'Unnamed: 28' ,
                'Unnamed: 29' , 'Unnamed: 30','Unnamed: 31' ,
                'Unnamed: 32' ,'Prénom','Sexe','Nom','N°Panneau']

id = 'CodeINSEE'

col_to_keep1 , to_holes1 = get_analyse(res_train_df,id,col_to_drop1)

colonne identifiant : 'CodeINSEE' 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 colonnes avec > 60 % de valeurs manquantes 
[]

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

les colonnes à supprimer sont :
['Unnamed: 27', 'Unnamed: 26', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Prénom', 'Sexe', 'Nom', 'N°Panneau']  
et  [] 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


on conserve les colonnes  :
['CodeINSEE', 'Libellé du département', 'Libellé de la commune', 'Etat saisie', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins', 'Blancs',

In [59]:
# @title conclusion
write_markdown_conclusion(id, to_holes1, col_to_drop1, col_to_keep1)


#### Conclusion

* _Identifiant_ : ``'CodeINSEE'``

* _Colonnes avec plus de 60% de valeurs manquantes_ : ``[]``

* _Colonnes à supprimer_ : ``['Unnamed: 27', 'Unnamed: 26', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Prénom', 'Sexe', 'Nom', 'N°Panneau']``

* _Colonnes à conserver_ : ``['CodeINSEE', 'Libellé du département', 'Libellé de la commune', 'Etat saisie', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins', '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot', 'Voix', '% Voix/Ins', '% Voix/Exp']``

----


#### conclusion

*  identifiant ``CodeINSEE``
*  colonnes avec trop de valeurs manquantes ` `
*  colonnes à évidenment supprimer ` 'Unnamed: 27', 'Unnamed: 26', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Prénom', 'Sexe', 'Nom', 'N°Panneau' `
*  les colonnes à conserver ` 'CodeINSEE', 'Libellé du département', 'Libellé de la commune', 'Etat saisie', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins', '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot', 'Voix', '% Voix/Ins', '% Voix/Exp'`

----

### niveau de vie

In [60]:
# @title visualisation
visualise(niveau_vie_df)

niveau_vie_df.head(3)

forme : (36572, 4)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



,Code Commune,Nom Commune,Niveau de vie Commune,Niveau de vie Département
0,05047,Éourres,10021.250,19202.516
1,26142,Glandage,10215.000,19300.504
2,11317,Rodome,10908.500,17599.733


In [61]:
all_col_niveau_vie = all_columns(niveau_vie_df)


les colonnes sont : 
Index(['Code Commune', 'Nom Commune', 'Niveau de vie Commune',
       'Niveau de vie Département'],
      dtype='object')

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 il y'a 4 colonnes dans le dataframe

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [62]:
# @title présélection
# colonnes clairement non informatives à supprimer
col_to_drop2 = []
id2 = 'Code Commune'

col_to_keep2 , to_holes2 = get_analyse(niveau_vie_df,id2,col_to_drop2)

colonne identifiant : 'Code Commune' 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 colonnes avec > 60 % de valeurs manquantes 
[]

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

les colonnes à supprimer sont :
[]  
et  [] 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


on conserve les colonnes  :
['Code Commune', 'Nom Commune', 'Niveau de vie Commune', 'Niveau de vie Département'] 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [63]:
# @title conclusion
write_markdown_conclusion(id2, to_holes2, col_to_drop2, col_to_keep2)


#### Conclusion

* _Identifiant_ : ``'Code Commune'``

* _Colonnes avec plus de 60% de valeurs manquantes_ : ``[]``

* _Colonnes à supprimer_ : ``[]``

* _Colonnes à conserver_ : ``['Code Commune', 'Nom Commune', 'Niveau de vie Commune', 'Niveau de vie Département']``

----


### communes de frances

In [64]:
# @title visualisation
visualise(communes_df)

communes_df.head(3)

forme : (35010, 39)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



,Unnamed: 0,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,reg_code,reg_nom,dep_code,dep_nom,canton_code,canton_nom,epci_code,epci_nom,academie_code,academie_nom,code_postal,codes_postaux,zone_emploi,code_insee_centre_zone_emploi,population,superficie_hectare,superficie_km2,densite,altitude_moyenne,altitude_minimale,altitude_maximale,latitude_mairie,longitude_mairie,latitude_centre,longitude_centre,grille_densite,gentile,url_wikipedia,url_villedereve
0,0,01001,L'Abergement-Clémenciat,Abergement-Clémenciat,à Abergement-Clémenciat,de l'Abergement-Clémenciat,l-abergement-clemenciat,L'ABERGEMENT-CLÉMENCIAT,COM,commune,84,Auvergne-Rhône-Alpes,01,Ain,0108,Châtillon-sur-Chalaronne,200069193,CC de la Dombes,10,Lyon,1400,01400,8405.000,01053,779,1565,16,48.700,242,206.000,272.000,46.153,4.926,46.153,4.926,Rural à habitat dispersé,NaN,https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01001-l-abergemen...
1,1,01002,L'Abergement-de-Varey,Abergement-de-Varey,à Abergement-de-Varey,de l'Abergement-de-Varey,l-abergement-de-varey,L'ABERGEMENT-DE-VAREY,COM,commune,84,Auvergne-Rhône-Alpes,01,Ain,0101,Ambérieu-en-Bugey,240100883,CC de la Plaine de l'Ain,10,Lyon,1640,01640,8405.000,01053,256,912,9,27.100,483,290.000,748.000,46.009,5.428,46.009,5.428,Rural à habitat dispersé,"Abergementais, Abergementaises",https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01002-l-abergemen...
2,2,01004,Ambérieu-en-Bugey,Ambérieu-en-Bugey,à Ambérieu-en-Bugey,d'Ambérieu-en-Bugey,amberieu-en-bugey,AMBÉRIEU-EN-BUGEY,COM,commune,84,Auvergne-Rhône-Alpes,01,Ain,0101,Ambérieu-en-Bugey,240100883,CC de la Plaine de l'Ain,10,Lyon,1500,"01500, 01501, 01504, 01503, 01502, 01505, 01506",8405.000,01053,14134,2448,24,570.500,379,237.000,753.000,45.961,5.373,45.961,5.373,Centres urbains intermédiaires,"Ambarrois, Ambarroises",https://fr.wikipedia.org/wiki/fr:Ambérieu-en-B...,https://villedereve.fr/ville/01004-amberieu-en...


In [68]:
# @title all colonnes
all_col_commune = all_columns(communes_df)

les colonnes sont : 
Index(['Unnamed: 0', 'code_insee', 'nom_standard', 'nom_sans_pronom', 'nom_a',
       'nom_de', 'nom_sans_accent', 'nom_standard_majuscule', 'typecom',
       'typecom_texte', 'reg_code', 'reg_nom', 'dep_code', 'dep_nom',
       'canton_code', 'canton_nom', 'epci_code', 'epci_nom', 'academie_code',
       'academie_nom', 'code_postal', 'codes_postaux', 'zone_emploi',
       'code_insee_centre_zone_emploi', 'population', 'superficie_hectare',
       'superficie_km2', 'densite', 'altitude_moyenne', 'altitude_minimale',
       'altitude_maximale', 'latitude_mairie', 'longitude_mairie',
       'latitude_centre', 'longitude_centre', 'grille_densite', 'gentile',
       'url_wikipedia', 'url_villedereve'],
      dtype='object')

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 il y'a 39 colonnes dans le dataframe

--------------------------

A première vu , on n'a pas besoin des
* url vers les sites internet des communes  c'est à dire ``url_wikipedia`` , ``url_ville``
* ``typecom`` et ``typecom_texte`` sont des colonnes constantes : on est toujours censé avoir à faire à des communes
* pas besoin de tous les type de noms de la communes , un seul suffira
* unamed ici represente un index donc inutile aussi

In [66]:
# @title présélection
# colonnes clairement non informatives à supprimer
col_to_drop3 = ['url_wikipedia','url_villedereve','typecom','typecom_texte',
                'nom_sans_pronom','nom_a','nom_de',
                'nom_sans_accent','nom_standard_majuscule',
                'superficie_hectare','Unnamed: 0']
id3 = 'code_insee'

col_to_keep3 , to_holes3 = get_analyse(communes_df,id3,col_to_drop3)

colonne identifiant : 'code_insee' 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 colonnes avec > 60 % de valeurs manquantes 
[]

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

les colonnes à supprimer sont :
['url_wikipedia', 'url_villedereve', 'typecom', 'typecom_texte', 'nom_sans_pronom', 'nom_a', 'nom_de', 'nom_sans_accent', 'nom_standard_majuscule', 'superficie_hectare', 'Unnamed: 0']  
et  [] 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


on conserve les colonnes  :
['code_insee', 'nom_standard', 'reg_code', 'reg_nom', 'dep_code', 'dep_nom', 'canton_code', 'canton_nom', 'epci_

In [67]:
# @title conclusion
write_markdown_conclusion(id3, to_holes3, col_to_drop3, col_to_keep3)


#### Conclusion

* _Identifiant_ : ``'code_insee'``

* _Colonnes avec plus de 60% de valeurs manquantes_ : ``[]``

* _Colonnes à supprimer_ : ``['url_wikipedia', 'url_villedereve', 'typecom', 'typecom_texte', 'nom_sans_pronom', 'nom_a', 'nom_de', 'nom_sans_accent', 'nom_standard_majuscule', 'superficie_hectare', 'Unnamed: 0']``

* _Colonnes à conserver_ : ``['code_insee', 'nom_standard', 'reg_code', 'reg_nom', 'dep_code', 'dep_nom', 'canton_code', 'canton_nom', 'epci_code', 'epci_nom', 'academie_code', 'academie_nom', 'code_postal', 'codes_postaux', 'zone_emploi', 'code_insee_centre_zone_emploi', 'population', 'superficie_km2', 'densite', 'altitude_moyenne', 'altitude_minimale', 'altitude_maximale', 'latitude_mairie', 'longitude_mairie', 'latitude_centre', 'longitude_centre', 'grille_densite', 'gentile']``

----


### age-insee

In [72]:
# @title visualisation
visualise(age_df)

age_df.head(3)

forme : (34980, 26)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



,INSEE,NOM,EPCI,DEP,REG,F0-2,F3-5,F6-10,F11-17,F18-24,F25-39,F40-54,F55-64,F65-79,F80+,Unnamed: 15,H0-2,H3-5,H6-10,H11-17,H18-24,H25-39,H40-54,H55-64,H65-79,H80+
0,1001,L'Abergement-Clémenciat,200069193,D1,R84,13.414,12.509,19.214,37.182,14.062,70.119,84.512,60.032,64.311,20.164,NaN,18.070,14.403,34.540,40.257,14.231,72.498,81.849,61.039,55.240,18.353
1,1002,L'Abergement-de-Varey,240100883,D1,R84,2.994,6.050,12.232,11.869,5.202,20.498,33.975,12.365,12.419,7.027,NaN,2.994,6.116,6.953,22.349,6.394,19.540,37.479,10.977,15.687,8.879
2,1004,Ambérieu-en-Bugey,240100883,D1,R84,294.668,245.153,382.801,599.105,680.831,1451.111,1268.502,903.062,1064.992,517.133,NaN,256.304,289.985,485.793,613.182,669.385,1542.699,1238.120,782.771,750.040,252.364


In [73]:
# @title Présélection
col_to_drop4 = []
id4 = 'INSEE'

col_to_keep4 , to_holes4 = get_analyse(age_df,id4,col_to_drop4)
col_to_drop4 += to_holes4

colonne identifiant : 'INSEE' 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 colonnes avec > 60 % de valeurs manquantes 
['Unnamed: 15']

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

les colonnes à supprimer sont :
[]  
et  ['Unnamed: 15'] 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


on conserve les colonnes  :
['INSEE', 'NOM', 'EPCI', 'DEP', 'REG', 'F0-2', 'F3-5', 'F6-10', 'F11-17', 'F18-24', 'F25-39', 'F40-54', 'F55-64', 'F65-79', 'F80+', 'H0-2', 'H3-5', 'H6-10', 'H11-17', 'H18-24', 'H25-39', 'H40-54', 'H55-64', 'H65-79', 'H80+'] 

---------------------------------------------

In [74]:
# @title conclusion
write_markdown_conclusion(id4, to_holes4, col_to_drop4, col_to_keep4)


#### Conclusion

* _Identifiant_ : ``'INSEE'``

* _Colonnes avec plus de 60% de valeurs manquantes_ : ``['Unnamed: 15']``

* _Colonnes à supprimer_ : ``['Unnamed: 15']``

* _Colonnes à conserver_ : ``['INSEE', 'NOM', 'EPCI', 'DEP', 'REG', 'F0-2', 'F3-5', 'F6-10', 'F11-17', 'F18-24', 'F25-39', 'F40-54', 'F55-64', 'F65-79', 'F80+', 'H0-2', 'H3-5', 'H6-10', 'H11-17', 'H18-24', 'H25-39', 'H40-54', 'H55-64', 'H65-79', 'H80+']``

----


### MDB-INSEE-Divers

In [75]:
# @title visualisation
visualise(insee_divers_df)

insee_divers_df.head(3)

forme : (36677, 101)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



,CODGEO,Nb Pharmacies et parfumerie,Dynamique Entrepreneuriale,Dynamique Entrepreneuriale Service et Commerce,Synergie Médicale COMMUNE,Orientation Economique,Indice Fiscal Partiel,Score Fiscal,Indice Evasion Client,Score Evasion Client,Indice Synergie Médicale,Score Synergie Médicale,SEG Croissance POP,LIBGEO,REG,DEP,Nb Omnipraticiens BV,Nb Infirmiers Libéraux BV,Nb dentistes Libéraux BV,Nb pharmaciens Libéraux BV,Densité Médicale BV,Score équipement de santé BV,Indice Démographique,Score Démographique,Indice Ménages,Score Ménages,Population,Evolution Population,Evolution Pop %,Nb Ménages,Nb Résidences Principales,Nb propriétaire,Nb Logement,Nb Résidences Secondaires,Nb Log Vacants,Nb Occupants Résidence Principale,Nb Femme,Nb Homme,Nb Mineurs,Nb Majeurs,Nb Etudiants,Nb Entreprises Secteur Services,Nb Entreprises Secteur Commerce,Nb Entreprises Secteur Construction,Nb Entreprises Secteur Industrie,Nb Création Enteprises,Nb Création Industrielles,Nb Création Construction,Nb Création Commerces,Nb Création Services,Moyenne Revenus Fiscaux Départementaux,Moyenne Revenus Fiscaux Régionaux,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Dep Moyenne Salaires Ouvrié Horaires,Reg Moyenne Salaires Horaires,Reg Moyenne Salaires Cadre Horaires,Reg Moyenne Salaires Prof Intermédiaire Horaires,Reg Moyenne Salaires Employé Horaires,Reg Moyenne Salaires Ouvrié Horaires,Valeur ajoutée régionale,Urbanité Ruralité,Score Urbanité,Nb Atifs,Nb Actifs Salariés,Nb Actifs Non Salariés,Nb Logement Secondaire et Occasionnel,Nb Hotel,Capacité Hotel,Nb Camping,Capacité Camping,Dynamique Démographique BV,Taux étudiants,Taux Propriété,Dynamique Démographique INSEE,Capacité Fisc,Capacité Fiscale,Moyenne Revnus fiscaux,Taux Evasion Client,"Nb Education, santé, action sociale",Nb Services personnels et domestiques,"Nb Santé, action sociale",Nb Industries des biens intermédiaires,Nb de Commerce,Nb de Services aux particuliers,"Nb institution de Education, santé, action sociale, administration",PIB Régionnal,SEG Environnement Démographique Obsolète,Score Croissance Population,Score Croissance Entrepreneuriale,Score VA Région,Score PIB,Environnement Démographique,Fidélité,SYN MEDICAL,Seg Cap Fiscale,Seg Dyn Entre,DYN SetC,CP
0,01001,0.000,57.000,23.000,114,Bassin Industriel,101.939,59.041,0.000,0.000,114.567,0.135,en croissance démographique,L' Abergement-Clémenciat,82,01,9,14,7,7,0.093,4,44.198,0.034,37.220,0.022,725,16,2,247,248,196,289,32,9,728,694,714,909,499,51,7.000,11.000,2.000,2.000,4.000,0.000,2.000,1.000,1.000,12509,10458,11.410,21.964,12.559,8.743,9.269,11.873,21.788,12.704,8.783,9.301,86957.458,Com rurale < 2 000 m habts,0.000,295.000,254.000,41.000,32.000,0.000,0.000,0.000,0.000,1.Accroissement par excédent naturel et migrat...,0,67,-1,117,117,11483.500,0,3.000,1.000,0.000,9364,9350,3372,15105,173681,Zone rurale en croissance démographique,72.131,0.016,32.426,33.838,Bassin Industriel en croissance démographique,Pop Sédentaire,Synergie Médicale,Fiscalité moyenne,Faible dynamique,Faible Dynamique Serv et Com,01
1,01002,0.000,45.000,4.000,143,Bassin Résidentiel,101.939,59.041,0.000,0.000,143.711,0.174,en croissance démographique,L' Abergement-de-Varey,82,01,31,36,18,18,0.099,4,10.181,0.008,10.096,0.006,167,4,2,67,67,61,142,71,4,168,162,164,202,124,5,4.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,1.000,12509,10458,11.410,21.964,12.559,8.743,9.269,11.873,21.788,12.704,8.783,9.301,86957.458,Com rurale < 2 000 m habts,0.000,57.000,49.000,8.000,71.000,0.000,0.000,0.000,0.000,1.Accroissement par excédent naturel et migrat...,0,42,0,110,110,11483.500,0,0.000,0.000,0.000,9364,9350,3372,15105,173681,Zone rurale en croissance démographique,72.131,0.002,32.426,33.838,Bassin Résidentiel en croissance démographique,Pop Sédentaire,Forte Synergie Médicale,Fiscalité moyenne,Faible dynamique,Faible Dynamique Serv et Com,01
2,01004,0.000,634.000,828.000,366,Bassin R

* pas besoin du `Nb Résidences Principales` et `Nb Résidences Secondaires` , `Nb log Vacants`  car leur somme = `Nb Logement`
* on peut regrouper les colonnes `Moyenne Revenus Fiscaux Départementaux` à `Moyenne Revenus Fiscaux Régionaux` en `indice fiscal` par  Indice_fiscal_dep $= \frac{Moyenne Revenus Fiscaux Départementaux}{Moyenne Revenus Fiscaux Régionaux}$
* regrouper  `Dep Moyenne Salaires Horaires` à `Reg Moyenne Salaires Horaires`  en indice salariaux -> utiliser la méthodes ACP apprament qui permet de les réduire à 2 indices synthétiques `CP1` et `CP2`

In [76]:
# @title affichage des colonnes
all_col_insee_divers = all_columns(insee_divers_df)

les colonnes sont : 
Index(['CODGEO', 'Nb Pharmacies et parfumerie', 'Dynamique Entrepreneuriale',
       'Dynamique Entrepreneuriale Service et Commerce',
       'Synergie Médicale COMMUNE', 'Orientation Economique',
       'Indice Fiscal Partiel', 'Score Fiscal', 'Indice Evasion Client',
       'Score Evasion Client',
       ...
       'Score Croissance Entrepreneuriale', 'Score VA Région', 'Score PIB',
       'Environnement Démographique', 'Fidélité', 'SYN MEDICAL',
       'Seg Cap Fiscale', 'Seg Dyn Entre', 'DYN SetC', 'CP'],
      dtype='object', length=101)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 il y'a 101 colonnes dans le dataframe

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [77]:
# @title présélection
id5 = 'CODGEO'
col_to_drop5 = []

col_to_keep5 , to_holes5 = get_analyse(insee_divers_df,id5,col_to_drop5)


colonne identifiant : 'CODGEO' 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 colonnes avec > 60 % de valeurs manquantes 
[]

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

les colonnes à supprimer sont :
[]  
et  [] 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


on conserve les colonnes  :
['CODGEO', 'Nb Pharmacies et parfumerie', 'Dynamique Entrepreneuriale', 'Dynamique Entrepreneuriale Service et Commerce', 'Synergie Médicale COMMUNE', 'Orientation Economique', 'Indice Fiscal Partiel', 'Score Fiscal', 'Indice Evasion Client', 'Score Evasion Client', 'Indice Synergie Médicale', 'S

## 1.3 Fusion des sources fournies et nettoyage

In [81]:
# @title colonnes à supprimer , avec trop de valeurs manquantes et à conserver
cols_to_drop = col_to_drop1 + col_to_drop2 + col_to_drop3 + col_to_drop4 + col_to_drop5
print(f"Toutes les colonnes à supprimer : {cols_to_drop}")

to_holes = to_holes1 + to_holes2 + to_holes3 + to_holes4 + to_holes5
print(f"Toutes les colonnes avec trop de valeurs manquantes : {to_holes}")

cols_to_keep = col_to_keep1 + col_to_keep2 + col_to_keep3 + col_to_keep4 + col_to_keep5
print(f"Toutes les colonnes à conserver : {cols_to_keep}")

print(f"Tous les identifiants : {IDs}")

Toutes les colonnes à supprimer : ['Unnamed: 27', 'Unnamed: 26', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Prénom', 'Sexe', 'Nom', 'N°Panneau', 'url_wikipedia', 'url_villedereve', 'typecom', 'typecom_texte', 'nom_sans_pronom', 'nom_a', 'nom_de', 'nom_sans_accent', 'nom_standard_majuscule', 'superficie_hectare', 'Unnamed: 0', 'Unnamed: 15']
Toutes les colonnes avec trop de valeurs manquantes : ['Unnamed: 15']
Toutes les colonnes à conserver : ['CodeINSEE', 'Libellé du département', 'Libellé de la commune', 'Etat saisie', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins', '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot', 'Voix', '% Voix/Ins', '% Voix/Exp', 'Code Commune', 'Nom Commune', 'Niveau de vie Commune', 'Niveau de vie Département', 'code_insee', 'nom_standard', 'reg_code', 'reg_nom', 'dep_code', 'dep_nom', 'canton_code', 'canton_nom', 'epci_code', 'epci_nom', 'academie

In [78]:
def safe_feature_selection(X, selected_features):
    """
    Retourne les colonnes de X correspondant aux selected_features valides.
    """
    return X[[feat for feat in selected_features if feat in X.columns]]

# @title fonction de fusion des dataframes
def prepare_datasets(train_data, test_data, niveau_vie, communes_france, age_insee, insee_divers):
    """
    Fonction pour préparer et fusionner les datasets pour la modélisation
    """
    #Extraire les données de Macron
    train_features = train_data[train_data['Nom']=='MACRON'].copy()
    test_features = test_data.copy()


    # Assurer que CodeINSEE est au format string avec padding
    train_features['CodeINSEE'] = train_features['CodeINSEE'].astype(str).str.zfill(5)
    test_features['CodeINSEE'] = test_features['CodeINSEE'].astype(str).str.zfill(5)
    niveau_vie_df['Code Commune'] = niveau_vie_df['Code Commune'].astype(str).str.zfill(5)
    communes_df['code_insee'] = communes_df['code_insee'].astype(str).str.zfill(5)
    age_insee['INSEE'] = age_insee['INSEE'].astype(str).str.zfill(5)
    insee_divers['CODGEO'] = insee_divers['CODGEO'].astype(str).str.zfill(5)

    #---------------------------
    # Fusion avec niveau de vie
    #--------------------------

    train_features = pd.merge(train_features, niveau_vie, left_on='CodeINSEE', right_on='Code Commune' , how='left')
    test_features = pd.merge(test_features, niveau_vie, left_on='CodeINSEE', right_on='Code Commune', how='left')

    #----------------------------
    # Fusion avec communes_france
    #----------------------------

    train_features = pd.merge(train_features, communes_df, left_on='CodeINSEE', right_on='code_insee', how='left')
    test_features = pd.merge(test_features, communes_df, left_on='CodeINSEE', right_on='code_insee', how='left')

    #--------------------------------------
    # fusion avec age_insee
    #--------------------------------------
    train_features = pd.merge(train_features, age_df, left_on='CodeINSEE', right_on='INSEE', how='left')
    test_features = pd.merge(test_features, age_df, left_on='CodeINSEE', right_on='INSEE', how='left')

    #-------------------------
    # Fusion avec insee_divers
    #-------------------------
    train_features = pd.merge(train_features, insee_divers_df, left_on='CodeINSEE', right_on='CODGEO', how='left')
    test_features = pd.merge(test_features, insee_divers_df, left_on='CodeINSEE', right_on='CODGEO', how='left')

    # Extraction de la variable cible
    y_train = train_features['% Voix/Ins'].copy()
    train_features.drop(columns=['% Voix/Ins'], inplace=True)
    test_features.drop(columns=['% Voix/Ins'], inplace=True, errors='ignore')

    # elimination
    all_col = train_features.columns.tolist()
    col_to_keep = sub(all_col,cols_to_drop)
    train_features = safe_feature_selection( train_features,col_to_keep)
    test_features = safe_feature_selection(test_features,col_to_keep)
    # Suppression des colonnes dupliquées identifiants IDs
    drop_cols = ['INSEE', 'code_insee', 'CODGEO', 'Commune']

    train_features = train_features.drop([col for col in drop_cols if col in train_features.columns], axis=1)
    test_features = test_features.drop([col for col in drop_cols if col in test_features.columns], axis=1)

    return train_features, y_train, test_features



# 2. Prétraitement , Pipeline et Selection automatique de features

In [82]:
X, y , test_data = prepare_datasets(res_train_df, res_test_df, niveau_vie_df, communes_df, age_df, insee_divers_df)

In [83]:
X.head(3)
print(len(X.columns))
get_analyse(X , id=IDs , col_to_drop=cols_to_drop, res=False)

175
colonne identifiant : '['CodeINSEE', 'Code Commune', 'code_insee', 'INSEE', 'INSEE', 'CODGEO', [...]]' 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 colonnes avec > 60 % de valeurs manquantes 
[]

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

les colonnes à supprimer sont :
['Unnamed: 27', 'Unnamed: 26', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Prénom', 'Sexe', 'Nom', 'N°Panneau', 'url_wikipedia', 'url_villedereve', 'typecom', 'typecom_texte', 'nom_sans_pronom', 'nom_a', 'nom_de', 'nom_sans_accent', 'nom_standard_majuscule', 'superficie_hectare', 'Unnamed: 0', 'Unnamed: 15']  
et  [] 

---------------------------------------------------------------------------------------------

## 2.1 Création du pipelines de prétraitement

In [114]:
# @title Pipelines de prétraitement
def get_preprocessor(X):
  """
  crée un préprocesseur pour les données d'entrée
  """
  X=X.copy()

  # Convertir les booléens en strings
  bool_cols = X.select_dtypes(include=['bool','object']).columns
  X[bool_cols] = X[bool_cols].astype(object)

  # identification des variables numériques et catégorielles
  #bool_cols = X.select_dtypes(include=['bool']).columns # booléen
  num_col = X.select_dtypes(include=['int64', 'float64']).columns # numériques
  cat_col = X.select_dtypes(include=['object']).columns # catégorielles

  # Transfomer pour les booléens (convertion en numériques)
  """
  bool_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('to_int', FunctionTransformer(lambda x: x.astype(int)))
        ])
        """
  # Transformer pour les variables numériques
  num_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy='median')),
      ('scaler', RobustScaler())
  ])

  # Transformer pour les variables catégorielles
  cat_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy='most_frequent')),
      ('onehot', OneHotEncoder(handle_unknown='ignore'))
  ])

  # Préprocesseur complet
  preprocessor = ColumnTransformer(
      transformers=[
          #('bool', bool_transformer, bool_cols),
          ('num', num_transformer, num_col),
          ('cat', cat_transformer, cat_col)
      ])
  return preprocessor

## 2.2 Sélection automatique des features

In [115]:
# @title Selection grossière
def quasi_constant_features(df, threshold=0.01):
    """
    Retourne la liste des colonnes du dataframe de variance très faible :
    ces colonnes sont quasi consantes , elles n'apportent pas d'information utile pour la modélisation
    """
    selector = VarianceThreshold(threshold=threshold)

    # Appliquer seulement sur les colonnes numériques
    numerical_df = df.select_dtypes(include=['int64', 'float64'])
    selector.fit(numerical_df)

    quasi_constant_cols = numerical_df.columns[~selector.get_support()]
    filtered_columns = [col for col in df.columns if col not in quasi_constant_cols]
    return filtered_columns

def select_important_features(X, y):
    """
    Sélectionne les features importantes en utilisant LassoCV, RandomForest et XGBoost.
    Retourne une liste consolidée de colonnes importantes.
    """
    preprocessor = get_preprocessor(X)
    # LassoCV
    pipe1 = Pipeline([
        ('preprocessor', preprocessor),
        ('lasso', LassoCV(cv=5, random_state=RANDOM_STATE))
    ])
    pipe1.fit(X, y)
    lasso_cv = pipe1.named_steps['lasso']
    lasso_coef = pd.Series(lasso_cv.coef_, index=X.columns)
    lasso_top = lasso_coef[lasso_coef != 0].index.tolist()

    # RandomForest
    pipe2 = Pipeline([
        ('preprocessor', preprocessor),
        ('rf', RandomForestRegressor(random_state=RANDOM_STATE))
    ])
    pipe2.fit(X, y)
    rf = pipe2.named_steps['rf']
    rf_importance = pd.Series(rf.feature_importances_, index=X.columns)
    rf_top = rf_importance.sort_values(ascending=False).head(20).index.tolist()

    # XGBoost
    pipe3 = Pipeline([
        ('preprocessor', preprocessor),
        ('xgb', XGBRegressor(random_state=RANDOM_STATE, verbosity=0))
    ])
    pipe3.fit(X, y)
    xgb = pipe3.named_steps['xgb']
    xgb_importance = pd.Series(xgb.feature_importances_, index=X.columns)
    xgb_top = xgb_importance.sort_values(ascending=False).head(20).index.tolist()

    # Consolidation des features importantes
    important_features = list(set(lasso_top + rf_top + xgb_top))
    return important_features

selcted_features = select_important_features(X, y)
print(f"Les colonnes importantes sont : \n{selcted_features} \n il y'en a {len(selcted_features)}")

TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['bool', 'str']

In [ ]:
# @title Selection  plus fine
def get_highly_correlated_features(X, threshold=0.9):
    """
    Retourne une liste de colonnes hautement corrélées.
    """
    numericals = X.select_dtypes(include=['int64', 'float64']).columns
    X_num = X[numericals]
    # Calculer la matrice de corrélation
    corr_matrix = X_num.corr().abs()
    # On ne garde que la partie supérieure de la matrice de corrélation
    # pour éviter les doublons
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    # Trouver les colonnes avec une corrélation supérieure au seuil
    # et les ajouter à la liste
    highly_correlated_features = [column for column in upper.columns if any(upper[column] > threshold)]
    return highly_correlated_features

# Optimisation de la selection des features
def backward_stepwise_selection(model,X,y,preprocessor=None,int_features:list=None,min_features:int=None,rtol:float=0.001,cv:int=5):
  """
  fonction de selection plus fines des meilleurs features selon un modèle
  """
  if int_features is None:
    features = list(X.columns)
  else:
    features = int_features

  if min_features is None:
    min_features = max( 5 , np.floor(0.1*len(features)) )

  if preprocessor is None:
    preprocessor = get_preprocessor(X)

  best_rmse = float('inf')

  while len(features) > min_features:
        worst_rmse = float('inf')
        worst_feature = None
        current_rmse = []

        # Évaluation de chaque feature
        for f in features:
            temp_features = [v for v in features if v != f]
            temp_model = Pipeline([
                ('preprocessor', preprocessor),
                ('model', model)
            ])
            temp_model.fit(X[temp_features], y)
            scores = cross_val_score(temp_model, X[temp_features], y,
                                    scoring='neg_root_mean_squared_error', cv=cv)
            rmse = -scores.mean()
            current_rmse.append(rmse)

            if rmse < worst_rmse:
                worst_rmse = rmse
                worst_feature = f

        relative_gain = (best_rmse - worst_rmse) / best_rmse
        current_rmse = np.array(current_rmse)

        # Critère d'arrêt dynamique
        if relative_gain < rtol :
            break

        if worst_rmse < best_rmse:
            best_rmse = worst_rmse
            features.remove(worst_feature)

        else:
            break

  return features


In [ ]:
correleted_features = get_highly_correlated_features(X)
print(f"Les colonnes hautement corrélées sont : \n{correleted_features} \n il y'en a {len(correleted_features)}")

Les colonnes hautement corrélées sont : ['Abstentions', 'Votants', '% Vot/Ins', 'Blancs', '% Blancs/Vot', '% Nuls/Vot', 'Exprimés', 'Voix', 'zone_emploi', 'population', 'altitude_maximale', 'latitude_centre', 'longitude_centre', 'Dynamique Entrepreneuriale', 'Dynamique Entrepreneuriale Service et Commerce', 'Synergie Médicale COMMUNE', 'Score Fiscal', 'Score Evasion Client', 'Indice Synergie Médicale', 'Score Synergie Médicale', 'REG', 'Nb dentistes Libéraux BV', 'Nb pharmaciens Libéraux BV', 'Indice Démographique', 'Score Démographique', 'Indice Ménages', 'Score Ménages', 'Population', 'Nb Ménages', 'Nb Résidences Principales', 'Nb propriétaire', 'Nb Logement', 'Nb Log Vacants', 'Nb Occupants Résidence Principale', 'Nb Femme', 'Nb Homme', 'Nb Mineurs', 'Nb Majeurs', 'Nb Etudiants', 'Nb Entreprises Secteur Services', 'Nb Entreprises Secteur Commerce', 'Nb Entreprises Secteur Construction', 'Nb Entreprises Secteur Industrie', 'Nb Création Enteprises', 'Nb Création Industrielles', 'Nb Cr

# 3. MODELISATION

### Fonction d'optimisation générale pour chaque modèle


In [ ]:

def optimise_model(model,params, X, y):
  """
  Ooptimise les hyperparamètres et selectionne les features optimales pour le modèle

  Args
  ---
  model : (fit_Object)
    instance du modèle choisit , ici ce sera soit Lasso , ElesticNet ,  XGboost , LightGBM
  params :
    disctionnaire permettant de definir l'espace des hyperparamètres à optimiser
  X :
    dataframes des predicteurs
  y :
    variables cible
  """
  cv = RepeatedKFold(n_splits=5,n_repeats=3,random_state=RANDOM_STATE)

  def objective(trial):
    # Définir les espaces de recherche pour chaque hyperparamètre
    param_grid = {}
    for name, conf in params.items():
      if conf['type'] == 'int':
        param_grid[name] = trial.suggest_int(name, *conf['range'])
      elif conf['type'] == 'float':
        param_grid[name] = trial.suggest_float(name, *conf['range'])
      elif conf['type'] == 'categorical':
        param_grid[name] = trial.suggest_categorical(name, conf['range'])

    # Appliquer les paramètres
    model.set_params(**param_grid)

    # Selection des features
    prepro = get_preprocessor(X)
    pipeline = Pipeline([
      ('preprocessor', prepro),
      ('model', model)
    ])

    selected_features = backward_stepwise_selection(pipeline, X, y, cv=cv)


    #sélection sécurisée des colonnes
    X_selected = safe_feature_selection(X, selected_features)

    final_prepro = get_preprocessor(X_selected)
    final_pipeline = Pipeline([
      ('preprocessor', final_prepro),
      ('model', model)
    ])

    trial.set_user_attr('selected_features', selected_features)

    # Effectuer la validation croisée
    scores = cross_val_score(final_pipeline,
                             X_selected, y,
                             cv=cv, scoring='neg_root_mean_squared_error')
    return -np.mean(scores)

  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

  best_params = study.best_params
  best_features = study.best_trial.user_attrs['selected_features']
  return best_params, best_features , study




## 3.2 Modèle imposé : Lasso


### initialisation du modèle

In [ ]:
lasso_model = Lasso(random_state=RANDOM_STATE)

### définition des hyperparmètres à optimiser

In [ ]:
lasso_space = {
       'alpha': {'type': 'float', 'range': (0.001, 10)},
       'max_iter': {'type': 'int', 'range': (1000, 10000)},
       'fit_intercept': {'type': 'categorical', 'range': [True, False]}
   }

### Recherche des meilleurs hyperparamètres

In [ ]:
lasso_best_params, lasso_selected_features, lasso_study = optimise_model(lasso_model, X_train, y_train)

### Finalisation du modèle et évaluation

In [ ]:
X_train_lasso = X_train[lasso_selected_features]
X_test_lasso = X_test[lasso_selected_features]

# le modèle avec les hyperparamètres optimaux
lasso_model.set_params(**lasso_best_params)
lasso_pipeline = Pipeline([
    ('preprocessor', get_preprocessor(X_train_lasso)),
    ('model', lasso_model)
])

# fit sur l'ensemble d'entrainement
lasso_pipeline.fit(X_train_lasso, y_train)
# prediction sur l'ensemble de validation
y_pred_lasso = lasso_pipeline.predict(X_test_lasso)

## 3.3 ElasticNet ( modèl choisit 1 )

### initialisation du modèle

In [ ]:
Enet_model = ElasticNet(random_state=RANDOM_STATE)

### définition des hyperparmètres à optimiser

In [ ]:
Enet_space = {
       'alpha': {'type': 'float', 'range': (0.001, 10)},
       'l1_ratio': {'type': 'float', 'range': (0.1, 1)},
       'max_iter': {'type': 'int', 'range': (1000, 10000)},
       'fit_intercept': {'type': 'categorical', 'range': [True, False]}
   }

### Recherche des meilleurs hyperparamètres

In [ ]:
Enet_best_params, Enet_selected_features, Enet_study = optimise_model(Enet_model, X_train, y_train)

### Finalisation du modèle et évaluation

In [ ]:
X_train_enet = X_train[Enet_selected_features]
X_test_enet = X_test[Enet_selected_features]

# le modèle avec les hyperparamètres optimaux
Enet_model.set_params(**Enet_best_params)
enet_pipeline = Pipeline([
    ('preprocessor', get_preprocessor(X_train_enet)),
    ('model', Enet_model)
])
# fit sur le train set
enet_pipeline.fit(X_train_enet, y_train)

# prédiction sur l'ensemble de validation
y_pred_enet = enet_pipeline.predict(X_test_enet)

## 3.4 XGboosting ( modèl choisit 2 )

### initialisation du modèle

In [ ]:
xgb_model = XGBRegressor(random_state=RANDOM_STATE)

### définition des hyperparmètres à optimiser

In [ ]:
xgb_space = {
    'n_estimators': {'type': 'int', 'range': (100, 1000)},       # Number of boosting rounds (trees)
    'max_depth': {'type': 'int', 'range': (3, 10)},             # Maximum depth of each tree
    'learning_rate': {'type': 'float', 'range': (0.01, 0.3)},   # Step size shrinkage used in update to prevents overfitting
    'subsample': {'type': 'float', 'range': (0.5, 1.0)},        # Subsample ratio of the training instance
    'colsample_bytree': {'type': 'float', 'range': (0.5, 1.0)},  # Subsample ratio of columns when constructing each tree
    'gamma': {'type': 'float', 'range': (0, 5)},                # Minimum loss reduction required to make a further partition on a leaf node of the tree
    'reg_alpha': {'type': 'float', 'range': (0, 1)},            # L1 regularization term on weights
    'reg_lambda': {'type': 'float', 'range': (0, 1)}             # L2 regularization term on weights
}

### Recherche des meilleurs hyperparamètres

In [ ]:
xgb_best_params, xgb_selected_features, xgb_study = optimise_model(xgb_model, xgb_space, X_train, y_train)

### Finalisation du modèle et évaluation

In [ ]:
X_train_xgb = X_train[xgb_selected_features]
X_test_xgb = X_test[xgb_selected_features]

# le modèle avec les hyperparamètres optimaux
xgb_model.set_params(**xgb_best_params)
xgb_pipeline = Pipeline([
    ('preprocessor', get_preprocessor(X_train_xgb)),
    ('model', xgb_model)
])

# fit sur l'ensemble d'entrainement
xgb_pipeline.fit(X_train_xgb, y_train)
# prediction sur l'ensemble de validation
y_pred_xgb = xgb_pipeline.predict(X_test_xgb)

## 2.4 Comparaison des modèles et selction du meilleur modèle

In [ ]:
# Fonction de compraison automatique
def compare_models(models_info, X_train, y_train, X_test, y_test):
    """
    Compare plusieurs modèles de régression en utilisant leurs meilleures features et hyperparamètres.

    Args:
    models_info (dict): Dictionnaire avec pour chaque clé un nom de modèle, et comme valeur :
        {
            "model": modèle instancié (ex: LinearRegression(), Lasso(), etc.),
            "params": meilleurs hyperparamètres,
            "features": liste des features optimales
        }
    X_train, y_train: données d'entraînement
    X_test, y_test: données de test

    Returns
    -------
    results (DataFrame):
        tableau comparatif des performances des modèles
    pipelines (dict):
        dictionnaire des pipelines entraînés pour réutilisation ultérieure
    """

    results = []
    pipelines = {}

    for name, info in models_info.items():
        model = info["model"]
        params = info["params"]
        features = info["features"]

        # Appliquer les meilleurs hyperparamètres
        model.set_params(**params)

        # Créer le préprocesseur
        preprocessor = get_preprocessor(X_train[features])

        # Construire le pipeline
        pipeline = Pipeline([
            ("preprocessor", preprocessor),
            ("model", model)
        ])

        # Entraîner le pipeline sur les meilleures features
        pipeline.fit(X_train[features], y_train)

        # Prédire sur le test
        y_pred = pipeline.predict(X_test[features])

        # Calculer les métriques
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        results.append({
            "Model": name,
            "RMSE": rmse,
            "MAE": mae,
            "R²": r2
        })

        # Sauvegarder le pipeline pour prédictions futures
        pipelines[name] = pipeline

    return pd.DataFrame(results).sort_values(by="RMSE"), pipelines

def save_model(model, filename):
    """
    Sauvegarde le modèle entraîné dans un fichier
    """
    joblib.dump(model, filename)


models_info = {
    "Lasso": {
        "model": Lasso(),
        "params": lasso_best_params,
        "features": lasso_selected_features
    },
    "ElasticNet": {
        "model": ElasticNet(),
        "params": Enet_best_params,
        "features": Enet_selected_features
    },
    "XGBoost": {
        "model": XGBRegressor(),
        "params": xgb_best_params,
        "features": xgb_selected_features
    }
}

results , pipelines = compare_models(models_info,
                                     X_train, y_train,
                                     X_test, y_test)

print(results)
results.sort_values(by='RMSE')
best_model_name = results.iloc[0]['Model']
best_model = models_info[best_model_name]['model']
best_pipeline = pipelines[best_model_name]
print(f"Le meilleur modèle selon la RMSE est le : {best_model}")

# 4. Soumission sur Kaggle

In [ ]:
# copier le fichier de test déja merge
data_test = test_data.copy()

# selectionner les features optimaux pour le best_model
data_test = data_test[models_info[best_model_name]['features']]

y_pred_test = best_pipeline.predict(data_test)

# Ajout des prédictions au dataframe
data_test['Prediction'] = y_pred_test

# sauvegarde du fichier
submission = data_test[['CodeINSEE', 'Prediction']]

submission.to_csv('results_test_predicted.csv', index=False)
print(f"\nFichier de soumission 'results_test_predicted.csv' généré avec succès \nForme {submission.shape}  ")

submission.head(5)